# Data Process Pipeline

### Data Process Commands

In [ ]:
import os

# TODO: change to your own project path!!!
OPEN_SORA_HOME = "/path/to/Open-Sora/"


def convert_dataset_cmd(input_dir, output_file, datatype="video"):
    commands = []
    commands.append(f'echo "Converting {input_dir} to {output_file}"')
    output_dir = os.path.dirname(output_file)

    commands.append(f"mkdir -p {output_dir}")
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(f"python -m tools.datasets.convert {datatype} {input_dir} --output {output_file}")
    return " && ".join(commands), output_file


def get_video_info(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_info{ext}"
    output_format = ext[1:]

    commands.append(f'echo "Getting info of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(
        f"python -m tools.datasets.datautil {input_file} --output {output_file} --format {output_format} --info --fmin 1"
    )
    return " && ".join(commands), output_file


def get_video_info_torchvision(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_info{ext}"
    output_format = ext[1:]

    commands.append(f'echo "Getting info of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(
        f"python -m tools.datasets.datautil {input_file} --output {output_file} --format {output_format} --video-info --fmin 1"
    )
    return " && ".join(commands), output_file


def get_caption_llava7b_video(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_caption{ext}"
    output_format = ext[1:]

    commands.append(f'echo "Getting info of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(f"conda activate llava2")
    commands.append(
        f"torchrun --nproc_per_node 8 --standalone -m tools.caption.caption_llava {input_file} --dp-size 8 --tp-size 1 --model-path liuhaotian/llava-v1.6-mistral-7b --prompt video"
    )
    commands.append(f"conda activate opensora")
    commands.append(
        f"python -m tools.datasets.datautil {base}_caption_part*{ext} --output {output_file} --format {output_format} --intersection {input_file} --clean-caption --refine-llm-caption --remove-empty-caption"
    )
    return " && ".join(commands), output_file


def get_caption_load(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_caption{ext}"
    output_format = ext[1:]

    commands.append(f'echo "Getting caption of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(
        f"python -m tools.datasets.datautil {input_file} --output {output_file} --format {output_format} --load-caption json --remove-empty-caption --clean-caption"
    )
    return " && ".join(commands), output_file


def get_aesthetic_score(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_aes{ext}"
    output_format = ext[1:]

    commands.append(f'echo "Getting aesthetic score of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(f"torchrun --standalone --nproc_per_node 8 -m tools.scoring.aesthetic.inference {input_file}")
    commands.append(
        f"python -m tools.datasets.datautil {base}_aes_part*{ext} --output {output_file} --format {output_format} --sort aes"
    )
    return " && ".join(commands), output_file


def get_flow_score(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_flow{ext}"

    commands.append(f'echo "Getting flow score of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(f"torchrun --standalone --nproc_per_node 8 -m tools.scoring.optical_flow.inference {input_file}")
    return " && ".join(commands), output_file


def get_ocr(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_match{ext}"

    commands.append(f'echo "Getting match score of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(f"torchrun --standalone --nproc_per_node 8 -m tools.scoring.ocr.inference {input_file}")
    return " && ".join(commands), output_file

    
def get_match_score(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_match{ext}"

    commands.append(f'echo "Getting match score of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(f"torchrun --standalone --nproc_per_node 8 -m tools.scoring.matching.inference {input_file}")
    return " && ".join(commands), output_file


def get_cmotion_score(input_file):
    commands = []
    base, ext = os.path.splitext(input_file)
    output_file = f"{base}_cmotion{ext}"

    commands.append(f'echo "Getting cmotion score of {input_file} to {output_file}"')
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append(f"python -m tools.caption.camera_motion_detect {input_file}")
    return " && ".join(commands), output_file


def get_commands(job_list):
    commands = []
    output_file = None
    for job in job_list:
        cmd = job.pop("cmd")
        if output_file is None:
            command, output_file = cmd(**job)
            commands.append(command)
        else:
            job["input_file"] = output_file
            command, output_file = cmd(**job)
            commands.append(command)
    commands.append(f'echo "All Done!"')
    return " && ".join(commands), output_file

### Remote Launch via Paramiko

First, add hosts to `~/.ssh/config`

In [ ]:
import paramiko

HOSTS = ["host-0", "host-1", "host-2", "host-3", "host-4", "host-5", "host-6", "host-7"]

# load from ~/.ssh/config
ssh_config = paramiko.SSHConfig()
user_config_file = os.path.expanduser("~/.ssh/config")
if os.path.exists(user_config_file):
    with open(user_config_file) as f:
        ssh_config.parse(f)


def get_ssh_config(hostname):
    # get the configuration for the host
    user_config = ssh_config.lookup(hostname)
    cfg = {
        "hostname": user_config["hostname"],
        "username": user_config["user"],
        "port": int(user_config["port"]),
        "key_filename": user_config["identityfile"],
    }
    return cfg


def connect(hostname):
    cfg = get_ssh_config(hostname)
    # connect
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(**cfg)
    return client


def run_command(command, hostname, nohup=False, log_file=None, sleep=None):
    client = connect(hostname)
    print("HOST:", hostname)
    if sleep:
        command = f"sleep {sleep}; {command}"
    command = f"bash -ic '{command}'"
    if log_file:
        command = f"{command} >> {log_file} 2>&1"
    if nohup:
        command = f"nohup {command} &"
    print("COMMAND:", command)
    stdin, stdout, stderr = client.exec_command(command, get_pty=False)

    stdout_str = stdout.read().decode()
    stderr_str = stderr.read().decode()
    if stdout_str:
        print("==== STDOUT ====\n", stdout_str)
    if stderr_str:
        print("==== STDERR ====\n", stderr_str)

    client.close()


def run_command_all_hosts(command, hosts=HOSTS):
    for hostname in hosts:
        run_command(command, hostname)

Here are tools to examine machine's status.

In [ ]:
def nvidia_smi(host):
    if host:
        run_command("nvidia-smi", host)
    else:
        run_command_all_hosts("nvidia-smi")


def nvitop(host=None):
    if host:
        run_command(f"/home/user/.local/bin/nvitop -1", host)
    else:
        run_command_all_hosts("/home/user/.local/bin/nvitop -1")


def ps(host=None, interest="python|sleep|torchrun|colossal", all=True):
    cmd = "ps aux" if all else "ps ux"
    if host:
        if interest is None:
            run_command(f"{cmd} | cat", host)
        else:
            run_command(f'{cmd} | cat | grep --color=never -E "{interest}"', host)
    else:
        if interest is None:
            run_command_all_hosts(f"{cmd} | cat")
        else:
            run_command_all_hosts(f'{cmd} | cat | grep --color=never -E "{interest}"')


def kill(pid, host):
    run_command(f"kill -KILL {pid}", host)


def pkill(interest, host):
    run_command(f'pkill -9 -f "{interest}"', host)

### Example

Remote launch via paramiko.

In [ ]:
sleep = None
run_command(cmd, host, log_file=log_file, nohup=True, sleep=sleep)
ps(host)

Using following commands to monitor the status of the jobs.

In [ ]:
ps()

In [ ]:
nvitop(host)

In [ ]:
kill(, host)

# Training

In [ ]:
def colossal_run(data_path, load_path=None):
    commands = []
    commands.append(f"cd {OPEN_SORA_HOME}")
    command = f"colossalai run --nproc_per_node 8 --hostfile hostfile scripts/train.py configs/opensora-v1-1/train/video.py --wandb True --data-path {data_path}"
    if load_path:
        command = f"{command} --load-path {load_path}"
    commands.append(command)
    cmd = " && ".join(commands)
    return cmd


def kill_all():
    commands = []
    commands.append(f"cd {OPEN_SORA_HOME}")
    commands.append('cat hostfile  | xargs -I "{}" ssh "{}" pkill -9 python')
    cmd = " && ".join(commands)
    return cmd

### Examples

In [ ]:
host = "host-0"
log_file = os.path.join(OPEN_SORA_HOME, "logs/train.log")
data_path = "/path/to/meta.csv"
cmd = colossal_run(data_path)
print(cmd)

In [ ]:
run_command(cmd, host, log_file=log_file, nohup=True)

In [ ]:
cmd = kill_all()
run_command(cmd, host)